In [1]:
import pandas as pd
import numpy as np

In [2]:
# Key file names
clinical_file = "data_clinical_sample.txt"
expression_file = "data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt"

print(f"Clinical file path: {clinical_file}")
print(f"Expression file path: {expression_file}")

Clinical file path: data_clinical_sample.txt
Expression file path: data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt


In [3]:
clinical_df = pd.read_csv(clinical_file, sep='\t', comment='#')

print("First rows of clinical data:")
print(clinical_df.head())

# Show the dimensions (rows, columns)
print(f"\nClinical data dimensions: {clinical_df.shape}")

# Show general information and data types
print("\nClinical data information:")
clinical_df.info()

First rows of clinical data:
     PATIENT_ID        SAMPLE_ID ONCOTREE_CODE    CANCER_TYPE  \
0  TCGA-3C-AAAU  TCGA-3C-AAAU-01           ILC  Breast Cancer   
1  TCGA-3C-AALI  TCGA-3C-AALI-01           IDC  Breast Cancer   
2  TCGA-3C-AALJ  TCGA-3C-AALJ-01           IDC  Breast Cancer   
3  TCGA-3C-AALK  TCGA-3C-AALK-01           IDC  Breast Cancer   
4  TCGA-4H-AAAK  TCGA-4H-AAAK-01           ILC  Breast Cancer   

                CANCER_TYPE_DETAILED                      TUMOR_TYPE  GRADE  \
0  Breast Invasive Lobular Carcinoma  Infiltrating Lobular Carcinoma    NaN   
1   Breast Invasive Ductal Carcinoma   Infiltrating Ductal Carcinoma    NaN   
2   Breast Invasive Ductal Carcinoma   Infiltrating Ductal Carcinoma    NaN   
3   Breast Invasive Ductal Carcinoma   Infiltrating Ductal Carcinoma    NaN   
4  Breast Invasive Lobular Carcinoma  Infiltrating Lobular Carcinoma    NaN   

  TISSUE_PROSPECTIVE_COLLECTION_INDICATOR  \
0                                      No   
1              

In [4]:
# Load expression data
expression_df = pd.read_csv(expression_file, sep='\t', comment='#')

print("First rows of expression data:")
print(expression_df.head())

print(f"\nExpression data dimensions: {expression_df.shape}")

# Show general information 
print("\nExpression data information:")
expression_df.info()

First rows of expression data:
  Hugo_Symbol  Entrez_Gene_Id  TCGA-3C-AAAU-01  TCGA-3C-AALI-01  \
0    UBE2Q2P2       100134869           0.9714           1.7813   
1     HMGB1P1           10357              NaN              NaN   
2   LOC155060          155060           5.1426           1.5334   
3    RNU12-2P           26823              NaN              NaN   
4        SSX9          280660          -0.0757           0.3125   

   TCGA-3C-AALJ-01  TCGA-3C-AALK-01  TCGA-4H-AAAK-01  TCGA-5L-AAT0-01  \
0           0.2971           0.6341           1.2442           1.0947   
1              NaN              NaN              NaN              NaN   
2           1.9422           1.7309           0.1608           0.2438   
3              NaN              NaN              NaN              NaN   
4          -0.0757          -0.0757          -0.0757          -0.0757   

   TCGA-5T-A9QA-01  TCGA-A1-A0SB-01  ...  TCGA-UL-AAZ6-01  TCGA-UU-A93S-01  \
0           0.2546           1.2376  ...         

In [5]:
# --- Process Clinical Data ---

# 1. Set SAMPLE_ID as index
# Check if 'SAMPLE_ID' exists before trying to set it as index
if 'SAMPLE_ID' in clinical_df.columns:
    clinical_df.set_index('SAMPLE_ID', inplace=True)
    print("Index of clinical_df set to 'SAMPLE_ID'.")
else:
    print("Error: Column 'SAMPLE_ID' not found in clinical_df.")

# 2. Explore the subtype column 
subtype_column = 'CANCER_TYPE_DETAILED' # Check if this is the correct name in your output
if subtype_column in clinical_df.columns:
    print(f"\nDistribution of subtypes in '{subtype_column}':")
    print(clinical_df[subtype_column].value_counts())

    # 3. Select only the subtype column
    clinical_target = clinical_df[[subtype_column]].copy()
    print("\nReduced clinical DataFrame (only with the subtype):")
    print(clinical_target.head())
    print(f"\nClinical target dimensions: {clinical_target.shape}")
else:
    print(f"\nError: Column '{subtype_column}' not found. Check the column names in the previous output.")

Index of clinical_df set to 'SAMPLE_ID'.

Distribution of subtypes in 'CANCER_TYPE_DETAILED':
CANCER_TYPE_DETAILED
Breast Invasive Ductal Carcinoma            780
Breast Invasive Lobular Carcinoma           201
Breast Invasive Carcinoma (NOS)              77
Breast Invasive Mixed Mucinous Carcinoma     17
Metaplastic Breast Cancer                     8
Invasive Breast Carcinoma                     1
Name: count, dtype: int64

Reduced clinical DataFrame (only with the subtype):
                              CANCER_TYPE_DETAILED
SAMPLE_ID                                         
TCGA-3C-AAAU-01  Breast Invasive Lobular Carcinoma
TCGA-3C-AALI-01   Breast Invasive Ductal Carcinoma
TCGA-3C-AALJ-01   Breast Invasive Ductal Carcinoma
TCGA-3C-AALK-01   Breast Invasive Ductal Carcinoma
TCGA-4H-AAAK-01  Breast Invasive Lobular Carcinoma

Clinical target dimensions: (1084, 1)


In [6]:
# --- Process Expression Data ---

# 1. Set Hugo_Symbol as index
if 'Hugo_Symbol' in expression_df.columns:
    expression_df.set_index('Hugo_Symbol', inplace=True)
    print("Index of expression_df set to 'Hugo_Symbol'.")

    # 2. Remove Entrez_Gene_Id column
    if 'Entrez_Gene_Id' in expression_df.columns:
        expression_df.drop(columns=['Entrez_Gene_Id'], inplace=True)
        print("Column 'Entrez_Gene_Id' removed.")
    else:
        print("Warning: Column 'Entrez_Gene_Id' not found for removal.")

    # 3. Check for duplicates in the index
    if expression_df.index.duplicated().any():
        print(f"\nWarning! Found {expression_df.index.duplicated().sum()} duplicate gene names in the index.")
        # For now, just warn. We might need to handle them later.
        # Optional: view duplicates: print(expression_df[expression_df.index.duplicated(keep=False)])
    else:
        print("\nNo duplicate gene names found in the index.")

    # 4. Transpose the DataFrame 
    expression_df_T = expression_df.T
    print("\nTransposed expression DataFrame (Samples x Genes):")
    print(expression_df_T.head())
    print(f"\nTransposed dimensions: {expression_df_T.shape}") # Should be (1082, 20471)

    # 5. Check for NaNs in the transposed DataFrame
    nan_count = expression_df_T.isnull().sum().sum()
    print(f"\nTotal number of NaN values in expression data: {nan_count}")
    if nan_count > 0:
        print("We will need to handle these NaNs in the next step.")

else:
    # This else corresponds to the initial check for 'Hugo_Symbol'
    print("Error: Column 'Hugo_Symbol' not found in expression_df.")

Index of expression_df set to 'Hugo_Symbol'.
Column 'Entrez_Gene_Id' removed.

Warning! Found 41 duplicate gene names in the index.

Transposed expression DataFrame (Samples x Genes):
Hugo_Symbol      UBE2Q2P2  HMGB1P1  LOC155060  RNU12-2P    SSX9  EZHIP  \
TCGA-3C-AAAU-01    0.9714      NaN     5.1426       NaN -0.0757    NaN   
TCGA-3C-AALI-01    1.7813      NaN     1.5334       NaN  0.3125    NaN   
TCGA-3C-AALJ-01    0.2971      NaN     1.9422       NaN -0.0757    NaN   
TCGA-3C-AALK-01    0.6341      NaN     1.7309       NaN -0.0757    NaN   
TCGA-4H-AAAK-01    1.2442      NaN     0.1608       NaN -0.0757    NaN   

Hugo_Symbol      EFCAB8  SRP14P1  LOC391343  TRIM75P  ...  ZWILCH   ZWINT  \
TCGA-3C-AAAU-01     NaN      NaN        NaN      NaN  ... -0.1901  0.2586   
TCGA-3C-AALI-01     NaN      NaN        NaN      NaN  ...  3.8368  0.3218   
TCGA-3C-AALJ-01     NaN      NaN        NaN      NaN  ... -0.7864  3.2995   
TCGA-3C-AALK-01     NaN      NaN        NaN      NaN  ... -0.30

In [7]:
# --- Align Samples ---

# 1. Find the common indices (SAMPLE_ID)
common_samples = clinical_target.index.intersection(expression_df_T.index)
print(f"Number of common samples: {len(common_samples)}")

# 2. Filter both DataFrames to keep only common samples and in the same order
clinical_aligned = clinical_target.loc[common_samples]
expression_aligned = expression_df_T.loc[common_samples]

# 3. Verify the aligned dimensions
print(f"\nAligned clinical dimensions: {clinical_aligned.shape}")
print(f"Aligned expression dimensions: {expression_aligned.shape}")

# Verify that the indices are identical
if clinical_aligned.index.equals(expression_aligned.index):
    print("\nSuccess! The indices of both DataFrames now match.")
else:
    print("\nError: Indices still do not match. Review previous steps.")

Number of common samples: 1082

Aligned clinical dimensions: (1082, 1)
Aligned expression dimensions: (1082, 20471)

Success! The indices of both DataFrames now match.


In [8]:
# --- Handle Duplicate Genes ---

expression_no_duplicates = expression_aligned.copy()

# Calculate the mean expression for each gene column
gene_means = expression_no_duplicates.mean(axis=0) # axis=0 calculates the mean per column (gene)

# Identify duplicate column names (genes)
duplicated_genes = expression_no_duplicates.columns[expression_no_duplicates.columns.duplicated(keep=False)]
# Only print if duplicates were actually found
if not duplicated_genes.empty:
    print(f"\nDuplicate gene names found: {duplicated_genes.unique().tolist()}")

genes_to_drop = []
# Iterate over the unique gene names that had duplicates
for gene in duplicated_genes.unique():
    # Get the columns corresponding to this duplicate gene
    duplicate_columns = expression_no_duplicates.columns[expression_no_duplicates.columns == gene]
    # Find the mean expression for these specific duplicate columns
    mean_values = gene_means[duplicate_columns]
    # Identify the column (among duplicates for this gene) with the highest mean
    column_to_keep = mean_values.idxmax()
    # Mark the other duplicate columns (for this specific gene) for removal
    columns_to_drop_for_gene = duplicate_columns.difference([column_to_keep])
    genes_to_drop.extend(columns_to_drop_for_gene.tolist())

# Only proceed if there are genes to drop
if genes_to_drop:
    # Remove the less informative duplicate columns
    # Use .loc with boolean indexing to select columns to keep
    # Keep columns NOT in genes_to_drop OR keep the first instance of any remaining duplicates (safety net)
    expression_no_duplicates = expression_no_duplicates.loc[:, ~expression_no_duplicates.columns.isin(genes_to_drop) | ~expression_no_duplicates.columns.duplicated(keep='first')]

    print(f"\nNumber of gene columns removed due to duplicates: {len(genes_to_drop)}")
else:
    print("\nNo duplicate gene columns needed removal based on mean expression.")

print(f"Expression dimensions after handling duplicates: {expression_no_duplicates.shape}") # Should have fewer or same columns as before

# Check if duplicates still remain 
if not expression_no_duplicates.columns.duplicated().any():
    print("Duplicate genes successfully handled (kept highest mean version).")
else:
    print("Error: Duplicate genes still remain after handling.")

# Overwrite the aligned DataFrame with the version without duplicates (or the original if none were removed)
expression_aligned = expression_no_duplicates


Duplicate gene names found: ['PALM2AKAP2', 'NEBL', 'LCOR', 'CC2D2B', 'SPANXC', 'PLEKHG7', 'CDSN', 'MIA2', 'ELMOD1', 'QSOX1', 'FUT1', 'SNAP47', 'NKAIN3', 'PSMB5', 'TBXT']

Number of gene columns removed due to duplicates: 2
Expression dimensions after handling duplicates: (1082, 20460)
Error: Duplicate genes still remain after handling.


/tmp/ipykernel_5794/3876591336.py:23: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  column_to_keep = mean_values.idxmax()
/tmp/ipykernel_5794/3876591336.py:23: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  column_to_keep = mean_values.idxmax()


In [9]:
# --- Check NaNs before Imputing ---

# Calculate NaN percentage per gene (column)
nan_percent_per_gene = expression_aligned.isnull().mean() * 100 # .mean() on booleans gives the proportion of True

# Calculate NaN percentage per sample (row)
nan_percent_per_sample = expression_aligned.isnull().mean(axis=1) * 100 # axis=1 for rows

print(f"\nSummary of % NaNs per gene:")
print(nan_percent_per_gene.describe())

print(f"\nSummary of % NaNs per sample:")
print(nan_percent_per_sample.describe())

# Decide threshold (optional)
threshold = 50
genes_to_remove = nan_percent_per_gene[nan_percent_per_gene > threshold].index
samples_to_remove = nan_percent_per_sample[nan_percent_per_sample > threshold].index

# Check genes exceeding the threshold
if not genes_to_remove.empty:
    print(f"\nGenes with > {threshold}% NaNs ({len(genes_to_remove)}): {genes_to_remove.tolist()}")
    # Note: Actual removal is commented out by default
    # expression_aligned.drop(columns=genes_to_remove, inplace=True) # Uncomment to remove
else:
    print(f"\nNo genes with > {threshold}% NaNs.")

# Check samples exceeding the threshold
if not samples_to_remove.empty:
    print(f"\nSamples with > {threshold}% NaNs ({len(samples_to_remove)}): {samples_to_remove.tolist()}")
    # Note: Actual removal is commented out by default
    # expression_aligned.drop(index=samples_to_remove, inplace=True) # Uncomment to remove
    # clinical_aligned = clinical_aligned.drop(index=samples_to_remove) # Important to align clinical data again if samples are removed!
else:
    print(f"\nNo samples with > {threshold}% NaNs.")

# Verify dimensions after removing (if uncommented above)
# print(f"\nFinal dimensions before imputing: {expression_aligned.shape}")
# if 'clinical_aligned' in locals() and samples_to_remove.empty == False: # Check if clinical_aligned exists and samples were potentially removed
#     print(f"Clinical dimensions before imputing: {clinical_aligned.shape}")


Summary of % NaNs per gene:
count    20460.000000
mean         2.179863
std         14.602908
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        100.000000
dtype: float64

Summary of % NaNs per sample:
count    1.082000e+03
mean     2.179863e+00
std      3.110062e-15
min      2.179863e+00
25%      2.179863e+00
50%      2.179863e+00
75%      2.179863e+00
max      2.179863e+00
dtype: float64

Genes with > 50% NaNs (446): ['HMGB1P1', 'RNU12-2P', 'EZHIP', 'EFCAB8', 'SRP14P1', 'LOC391343', 'TRIM75P', 'SPATA31B1P', 'REXO1L6P', 'LOC553137', 'HSPB1P1', 'PPBPP1', 'ANKRD20A20P', 'AMELY', 'BCORP1', 'BMS1P1', 'BPY2', 'LINC00596', 'C4orf50', 'CCL4L1', 'CD24', 'CDSN', 'CDY1', 'CDY1B', 'CDY2B', 'CSPG4P2Y', 'CT47A10', 'CT47A9', 'CTAG1B', 'CXorf49B', 'FAM226B', 'TXLNGY', 'DAD1P1', 'DAZ1', 'DAZ2', 'DAZ3', 'DAZ4', 'DDX3Y', 'DEFA1B', 'DEFB110', 'DEFB112', 'DEFB113', 'DEFB114', 'DEFB116', 'DEFB122', 'DEFB127', 'DEFB128', 'DEFB130A', 'PHF2P1', 'DMRTC1B', 'DUX

In [10]:
# ---  Remove Duplicates ---

# Check duplicates BEFORE
print(f"Columns BEFORE removing duplicates: {expression_aligned.shape[1]}")
print(f"Are there duplicates BEFORE: {expression_aligned.columns.duplicated().any()}")

# Remove duplicate columns, keeping the first occurrence
# The selector `~expression_aligned.columns.duplicated(keep='first')` keeps columns that are not duplicates,
# or if they are duplicates, it keeps only the first instance encountered.
expression_no_duplicates_revised = expression_aligned.loc[:,~expression_aligned.columns.duplicated(keep='first')]

# Check duplicates AFTER
print(f"\nColumns AFTER removing duplicates: {expression_no_duplicates_revised.shape[1]}")
print(f"Are there duplicates AFTER: {expression_no_duplicates_revised.columns.duplicated().any()}") # This should now be False

# Overwrite the DataFrame
expression_aligned = expression_no_duplicates_revised

Columnas ANTES de eliminar duplicados: 20460
Hay duplicados ANTES: True

Columnas DESPUÉS de eliminar duplicados: 20430
Hay duplicados DESPUÉS: False


In [10]:
# --- Remove Genes with > 50% NaNs ---

print(f"Number of genes originally identified for removal (>50% NaN): {len(genes_to_remove)}")

# Check how many of those genes are actually still present in the DataFrame's columns
# (some might have been removed in previous steps, e.g., duplicate handling)
cols_actually_in_df = genes_to_remove.intersection(expression_aligned.columns)
print(f"Of those, {len(cols_actually_in_df)} are present in the current DataFrame.")

# Proceed only if there are high-NaN genes still present to be removed
if not cols_actually_in_df.empty:
    # Remove the columns that both have >50% NaNs AND are still present
    expression_aligned.drop(columns=cols_actually_in_df, inplace=True)
    print(f"\nGenes with > 50% NaNs removed.")
    print(f"New expression dimensions: {expression_aligned.shape}")
else:
    # This happens if the genes identified previously were already removed (e.g., by duplicate removal)
    print("\nNo columns with > 50% NaNs found to remove (perhaps already removed during duplicate handling).")

Number of genes originally identified for removal (>50% NaN): 446
Of those, 446 are present in the current DataFrame.

Genes with > 50% NaNs removed.
New expression dimensions: (1082, 20010)


In [12]:
# --- Impute NaNs with KNNImputer ---
from sklearn.impute import KNNImputer

print(f"\nNumber of NaNs BEFORE imputing: {expression_aligned.isnull().sum().sum()}")

if expression_aligned.isnull().sum().sum() > 0:
    print("Starting KNN imputation (may take a few minutes)...")

    # Initialize the imputer
    # n_neighbors=5 is a common value, you can adjust it if necessary
    imputer = KNNImputer(n_neighbors=5)

    # Save column names and index before imputing, as KNNImputer returns a NumPy array
    original_index = expression_aligned.index
    original_columns = expression_aligned.columns

    # Apply the imputer (fit and transform the data)
    expression_imputed_array = imputer.fit_transform(expression_aligned)

    # Convert the resulting NumPy array back to a pandas DataFrame
    expression_imputed = pd.DataFrame(expression_imputed_array,
                                      index=original_index,
                                      columns=original_columns)

    print("KNN imputation completed.")
    print(f"Number of NaNs AFTER imputing: {expression_imputed.isnull().sum().sum()}")

    # Verify that there are no NaNs left after imputation
    if expression_imputed.isnull().sum().sum() == 0:
        print("Success! All NaNs have been imputed.")
        # Overwrite the aligned DataFrame with the imputed version for subsequent steps
        expression_aligned = expression_imputed
    else:
        print("Error: NaNs still remain after imputation.")

else:
    print("\nNo NaNs found to impute.")

# Verify the final dimensions of both aligned dataframes to ensure they match on samples (rows)
print(f"\nFinal expression dimensions (Samples x Genes): {expression_aligned.shape}")
print(f"Final clinical dimensions (Samples x Target): {clinical_aligned.shape}")


Number of NaNs BEFORE imputing: 0

No NaNs found to impute.

Final expression dimensions (Samples x Genes): (1082, 20010)
Final clinical dimensions (Samples x Target): (1082, 1)


In [13]:
# --- Step 12: EDA with Dimensionality Reduction ---
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import os # Import os module for directory creation

# Directory to save images
image_dir = "images"
os.makedirs(image_dir, exist_ok=True) 
print(f"Plots will be saved to the '{image_dir}' directory.")

# 1. Scale the expression data (important for PCA/UMAP)
# StandardScaler centers the data (mean=0) and scales to unit variance
print("Scaling expression data...")
scaler = StandardScaler()
# Assuming expression_aligned is Samples x Genes
expression_scaled = scaler.fit_transform(expression_aligned)
print("Scaling complete.")

# --- PCA ---
print("\nCalculating PCA...")
pca = PCA(n_components=10) # Calculate the first 10 components
expression_pca = pca.fit_transform(expression_scaled)
print("PCA calculated.")

# Create a DataFrame with PCA results and add the subtype
pca_df = pd.DataFrame(data=expression_pca,
                      columns=[f'PC{i+1}' for i in range(10)],
                      index=expression_aligned.index) # Use the same index (SAMPLE_ID)
# Join with the clinical subtype information
# Ensure clinical_aligned has the SAMPLE_ID as index and contains subtype_column
pca_df = pca_df.join(clinical_aligned)

# Variance explained by each component
explained_variance_ratio = pca.explained_variance_ratio_
print("\nVariance explained by the first 10 PCs:")
print(explained_variance_ratio)
print(f"Total variance explained by the first 10 PCs: {explained_variance_ratio.sum():.2f}")

# Visualize PC1 vs PC2
print(f"Generating PCA plot (saving to {image_dir}/pca_plot.png)...")
plt.figure(figsize=(12, 8)) # Adjusted size slightly for legend
sns.scatterplot(
    x="PC1", y="PC2",
    hue=subtype_column, # Color by subtype
    data=pca_df,
    alpha=0.7, # Transparency
    s=50 # Point size
)
plt.title('PCA of Gene Expression (PC1 vs PC2)')
plt.xlabel(f'PC1 ({explained_variance_ratio[0]*100:.2f}% variance)') # Added "variance"
plt.ylabel(f'PC2 ({explained_variance_ratio[1]*100:.2f}% variance)') 
plt.legend(title=subtype_column, bbox_to_anchor=(1.05, 1), loc='upper left') 
plt.grid(True)
plt.tight_layout(rect=[0, 0, 0.85, 1]) 
plt.savefig(os.path.join(image_dir, 'pca_plot.png'), bbox_inches='tight') 
plt.close() 
print("PCA plot saved.")


# --- UMAP --- 
print("\nCalculating UMAP...")
# n_components=2 for 2D plot, random_state for reproducibility
reducer = umap.UMAP(n_components=2, random_state=42)
expression_umap = reducer.fit_transform(expression_scaled) # Use scaled data
print("UMAP calculated.")

# Create DataFrame for UMAP results
umap_df = pd.DataFrame(data=expression_umap,
                       columns=['UMAP1', 'UMAP2'],
                       index=expression_aligned.index) # Use the same index
# Join with the clinical subtype information
umap_df = umap_df.join(clinical_aligned)

# Visualize UMAP1 vs UMAP2
print(f"Generating UMAP plot (saving to {image_dir}/umap_plot.png)...")
plt.figure(figsize=(12, 8)) # Adjusted size slightly for legend
sns.scatterplot(
    x="UMAP1", y="UMAP2",
    hue=subtype_column, # Color by subtype
    data=umap_df,
    alpha=0.7,
    s=50
)
plt.title('UMAP of Gene Expression (UMAP1 vs UMAP2)')
plt.xlabel('UMAP Component 1') 
plt.ylabel('UMAP Component 2') 
plt.legend(title=subtype_column, bbox_to_anchor=(1.05, 1), loc='upper left') 
plt.grid(True)
plt.tight_layout(rect=[0, 0, 0.85, 1]) 
plt.savefig(os.path.join(image_dir, 'umap_plot.png'), bbox_inches='tight') # Save the figure
plt.close() 
print("UMAP plot saved.")

Plots will be saved to the 'images' directory.
Scaling expression data...
Scaling complete.

Calculating PCA...
PCA calculated.

Variance explained by the first 10 PCs:
[0.08340746 0.05566725 0.03901887 0.02309684 0.01872063 0.01645889
 0.01271541 0.0119289  0.01048777 0.00893311]
Total variance explained by the first 10 PCs: 0.28
Generating PCA plot (saving to images/pca_plot.png)...
PCA plot saved.

Calculating UMAP...


/home/puff/anaconda3/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP calculated.
Generating UMAP plot (saving to images/umap_plot.png)...
UMAP plot saved.


In [15]:
from sklearn.model_selection import train_test_split
# --- Prepare Data for Modeling ---

# 1. Define initial X (features) and y (target)
X = expression_aligned # Our clean, aligned expression data (Samples x Genes)
y = clinical_aligned[subtype_column] # Our subtype column (target)

print(f"Initial X shape: {X.shape}")
print(f"Initial y shape: {y.shape}")
print("Initial subtype distribution:")
print(y.value_counts())

# 2. Identify and remove classes with fewer than 2 members (necessary for stratified split)
print("\nChecking for classes with < 2 members...")
counts = y.value_counts()
classes_to_remove = counts[counts < 2].index

if not classes_to_remove.empty:
    print(f"Removing samples from classes with < 2 members: {classes_to_remove.tolist()}")
    # Get the SAMPLE_IDs of the samples to remove
    samples_to_remove_idx = y[y.isin(classes_to_remove)].index

    # Remove the samples from both X and y
    X = X.drop(index=samples_to_remove_idx)
    y = y.drop(index=samples_to_remove_idx)

    print(f"Samples removed. New dimensions:")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print("\nNew subtype distribution:")
    print(y.value_counts())
else:
    print("No classes with fewer than 2 members found to remove.")


# --- GROUP OTHER MINORITY CLASSES ---
# This step helps prevent small classes from dominating evaluation or causing issues
print("\nChecking for other minority classes to group...")
counts_after_removal = y.value_counts()
# Define threshold: group classes with fewer than, e.g., 25 samples
threshold_grouping = 25
types_to_replace = counts_after_removal[counts_after_removal < threshold_grouping].index

if not types_to_replace.empty:
    print(f"\nGrouping subtypes with < {threshold_grouping} samples into 'Other_Subtype': {types_to_replace.tolist()}")
    # Replace these minority types with 'Other_Subtype' in the target variable y
    y = y.replace(types_to_replace.tolist(), 'Other_Subtype')
    print("\nSubtype distribution after grouping:")
    print(y.value_counts())
else:
    print(f"\nNo additional minority subtypes (< {threshold_grouping} samples) found to group.")
# --- End of grouping ---

# 3. Split into training and test sets 
print("\nSplitting data into training and testing sets (70/30)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, # Use the potentially modified X and y
    test_size=0.3,       # 30% for the test set
    random_state=42,     # For reproducibility
    stratify=y           # Ensure distribution of 'y' is similar in train/test
)

print("\nData split into training and test sets:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Verify stratification
print("\nSubtype distribution in y_train (%):")
print(y_train.value_counts(normalize=True) * 100)
print("\nSubtype distribution in y_test (%):")
print(y_test.value_counts(normalize=True) * 100)

Initial X shape: (1082, 20010)
Initial y shape: (1082,)
Initial subtype distribution:
CANCER_TYPE_DETAILED
Breast Invasive Ductal Carcinoma            780
Breast Invasive Lobular Carcinoma           201
Breast Invasive Carcinoma (NOS)              75
Breast Invasive Mixed Mucinous Carcinoma     17
Metaplastic Breast Cancer                     8
Invasive Breast Carcinoma                     1
Name: count, dtype: int64

Checking for classes with < 2 members...
Removing samples from classes with < 2 members: ['Invasive Breast Carcinoma']
Samples removed. New dimensions:
X shape: (1081, 20010)
y shape: (1081,)

New subtype distribution:
CANCER_TYPE_DETAILED
Breast Invasive Ductal Carcinoma            780
Breast Invasive Lobular Carcinoma           201
Breast Invasive Carcinoma (NOS)              75
Breast Invasive Mixed Mucinous Carcinoma     17
Metaplastic Breast Cancer                     8
Name: count, dtype: int64

Checking for other minority classes to group...

Grouping subtypes with

In [16]:
# --- Scale Features ---
# Initialize the scaler
scaler = StandardScaler()

print("Scaling training and testing data...")

X_train_scaled = scaler.fit_transform(X_train) 

# Transform the test data using the scaler fitted on the training data
X_test_scaled = scaler.transform(X_test) 

print("Data scaled.")
# Note: The results (X_train_scaled, X_test_scaled) are NumPy arrays, not Pandas DataFrames
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

Scaling training and testing data...
Data scaled.
X_train_scaled shape: (756, 20010)
X_test_scaled shape: (325, 20010)


In [19]:
# --- Train and Evaluate RandomForest ---
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# 1. Initialize the RandomForest model
# n_estimators=100 
# random_state=42 
# class_weight='balanced' can help with imbalanced classes (adjusts weights inversely proportional to class frequencies)
# n_jobs=-1 uses all available CPU cores for training
rf_model = RandomForestClassifier(n_estimators=100,
                                random_state=42,
                                class_weight='balanced',
                                n_jobs=-1)

# 2. Train the model with the scaled training data
print("\nTraining RandomForest model...")
rf_model.fit(X_train_scaled, y_train)
print("Training completed.")

# 3. Predict the subtypes on the test set
print("Making predictions on the test set...")
y_pred = rf_model.predict(X_test_scaled)

# 4. Evaluate the performance
print("Evaluating model performance...")
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy on the test set: {accuracy:.4f}")

# Detailed Classification Report (precision, recall, f1-score per class)
print("\nClassification Report:")
# zero_division=0 prevents warnings if a class in y_true has no predictions (unlikely here but good practice)

try:
    # Get unique labels sorted 
    class_labels_sorted = sorted(y_test.unique())
    report = classification_report(y_test, y_pred, zero_division=0, labels=class_labels_sorted)
    print(report)
except Exception as e:
    print(f"Could not generate detailed report: {e}")
    # Fallback to basic report if labels cause issues
    report = classification_report(y_test, y_pred, zero_division=0)
    print(report)


# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred, labels=class_labels_sorted) # Ensure consistent label order
print(cm) 

# Plot the confusion matrix
print(f"Generating Confusion Matrix plot (saving to {image_dir}/confusion_matrix_rf.png)...")
plt.figure(figsize=(10, 8)) # Adjust size 
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels_sorted, yticklabels=class_labels_sorted)
plt.xlabel('Predicted Label') 
plt.ylabel('True Label') 
plt.title('Confusion Matrix - RandomForest') 
plt.xticks(rotation=45, ha='right') # Rotate labels if they overlap
plt.yticks(rotation=0)
plt.tight_layout() # Adjust layout
plt.savefig(os.path.join(image_dir, 'confusion_matrix_rf.png'), bbox_inches='tight') 
plt.close() # Close the figure
print("Confusion Matrix plot saved.")


Training RandomForest model...
Training completed.
Making predictions on the test set...
Evaluating model performance...

Accuracy on the test set: 0.7938

Classification Report:
                                   precision    recall  f1-score   support

  Breast Invasive Carcinoma (NOS)       0.00      0.00      0.00        23
 Breast Invasive Ductal Carcinoma       0.79      0.98      0.88       234
Breast Invasive Lobular Carcinoma       0.82      0.47      0.60        60
                    Other_Subtype       0.00      0.00      0.00         8

                         accuracy                           0.79       325
                        macro avg       0.40      0.36      0.37       325
                     weighted avg       0.72      0.79      0.74       325


Confusion Matrix:
[[  0  21   2   0]
 [  0 230   4   0]
 [  0  32  28   0]
 [  0   8   0   0]]
Generating Confusion Matrix plot (saving to images/confusion_matrix_rf.png)...
Confusion Matrix plot saved.


In [20]:
# --- Feature Selection ---
from sklearn.feature_selection import SelectKBest, f_classif


# Number of features (genes) to select
# We'll start with 1000
k_best = 1000

print(f"Selecting the top {k_best} features using SelectKBest with f_classif...")

# Ensure X_train is the original DataFrame before scaling to get gene names
# f_classif computes the ANOVA F-value between label/feature for classification tasks.
selector = SelectKBest(score_func=f_classif, k=k_best)

# Fit the selector to the training data
# We need y_train to calculate the feature association with the target
selector.fit(X_train, y_train)

# Get the mask and names of the selected columns
selected_cols_mask = selector.get_support()
selected_genes = X_train.columns[selected_cols_mask]
print(f"{len(selected_genes)} genes were selected.")

# Filter original X_train and X_test DataFrames to keep only the selected features
X_train_selected = X_train.loc[:, selected_genes]
X_test_selected = X_test.loc[:, selected_genes]

print(f"\nNew dimensions after selection:")
print(f"X_train_selected shape: {X_train_selected.shape}")
print(f"X_test_selected shape: {X_test_selected.shape}")

# --- RE-SCALE the selected data ---
# Re-scale AFTER feature selection using only the selected features
print("\nRe-scaling selected features...")
scaler_selected = StandardScaler() # Initialize a new scaler instance

# Fit the new scaler ONLY on the selected training data and transform it
X_train_selected_scaled = scaler_selected.fit_transform(X_train_selected)

# Transform the selected test data using the scaler fitted above
X_test_selected_scaled = scaler_selected.transform(X_test_selected)
print("Re-scaling complete.")

Selecting the top 1000 features using SelectKBest with f_classif...
1000 genes were selected.

New dimensions after selection:
X_train_selected shape: (756, 1016)
X_test_selected shape: (325, 1016)

Re-scaling selected features...
Re-scaling complete.


In [22]:
# --- Re-train and Evaluate RandomForest (with selected features) ---

# (The code is almost identical to the last cell, only the X input variables change)

# 1. Initialize the RandomForest model 
rf_model_selected = RandomForestClassifier(n_estimators=100,
                                         random_state=42,
                                         class_weight='balanced',
                                         n_jobs=-1)

# 2. Train the model with the SELECTED and SCALED training data
print("\nTraining RandomForest model with selected features...")
rf_model_selected.fit(X_train_selected_scaled, y_train)
print("Training completed.")

# 3. Predict on the SELECTED and SCALED test set
print("Making predictions on the test set using selected features...")
# We use X_test_selected_scaled!!
y_pred_selected = rf_model_selected.predict(X_test_selected_scaled)

# 4. Evaluate the performance
print("Evaluating model performance with selected features...")
accuracy_selected = accuracy_score(y_test, y_pred_selected)
print(f"\nAccuracy with selected features: {accuracy_selected:.4f}")

# Detailed Classification Report
print("\nClassification Report (with selected features):")
# It's important to get the correct labels, especially if 'Other_Subtype' was created
try:
    class_labels_selected = sorted(y_test.unique())
    report_selected = classification_report(y_test, y_pred_selected, zero_division=0, labels=class_labels_selected)
    print(report_selected)
except Exception as e:
     print(f"Could not generate detailed report: {e}")
     report_selected = classification_report(y_test, y_pred_selected, zero_division=0)
     print(report_selected)


# Confusion Matrix
print("\nConfusion Matrix (with selected features):")
# Ensure label order consistency
cm_selected = confusion_matrix(y_test, y_pred_selected, labels=class_labels_selected)
print(cm_selected) # Print raw matrix

# Plot the confusion matrix
print(f"Generating Confusion Matrix plot (saving to {image_dir}/confusion_matrix_rf_selected.png)...")
plt.figure(figsize=(10, 8)) 
sns.heatmap(cm_selected, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels_selected, yticklabels=class_labels_selected)
plt.xlabel('Predicted Label') 
plt.ylabel('True Label') 
plt.title('Confusion Matrix - RandomForest (Selected Features)') 
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout() 
plt.savefig(os.path.join(image_dir, 'confusion_matrix_rf_selected.png'), bbox_inches='tight')
plt.close() 
print("Confusion Matrix plot saved.")


Training RandomForest model with selected features...
Training completed.
Making predictions on the test set using selected features...
Evaluating model performance with selected features...

Accuracy with selected features: 0.8031

Classification Report (with selected features):
                                   precision    recall  f1-score   support

  Breast Invasive Carcinoma (NOS)       0.00      0.00      0.00        23
 Breast Invasive Ductal Carcinoma       0.81      0.97      0.88       234
Breast Invasive Lobular Carcinoma       0.77      0.57      0.65        60
                    Other_Subtype       0.00      0.00      0.00         8

                         accuracy                           0.80       325
                        macro avg       0.40      0.38      0.38       325
                     weighted avg       0.72      0.80      0.76       325


Confusion Matrix (with selected features):
[[  0  20   3   0]
 [  0 227   7   0]
 [  0  26  34   0]
 [  0   8   0 

In [23]:
# --- Step 18: Apply SMOTE and Re-train ---
from imblearn.over_sampling import SMOTE

# 1. Initialize SMOTE
# KNN = 5 
smote = SMOTE(random_state=42, k_neighbors=5)
print("\nApplying SMOTE to the training set...")
# Check class distribution before applying SMOTE
print(f"Class distribution BEFORE SMOTE:\n{pd.Series(y_train).value_counts()}") # Ensure y_train is treated as Series for value_counts

# 2. Apply SMOTE ONLY to the training data
# SMOTE should not be applied to the test set
# fit_resample returns the oversampled X and y data
try:
    X_train_smote, y_train_smote = smote.fit_resample(X_train_selected_scaled, y_train)
    print("\nSMOTE applied successfully.")
    print(f"X_train shape AFTER SMOTE: {X_train_smote.shape}") 
    print(f"Class distribution AFTER SMOTE:\n{pd.Series(y_train_smote).value_counts()}") # All classes should have the same size (size of original majority class)

    # 3. Initialize and Train RandomForest with SMOTE data
    # (Using a new model variable name for clarity)
    rf_model_smote = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    print("\nTraining RandomForest model with SMOTE data...")
    # Train using the SMOTE-resampled training data!
    rf_model_smote.fit(X_train_smote, y_train_smote)
    print("Training completed.")

    # 4. Predict on the ORIGINAL test set (selected and scaled, but NOT resampled)
    print("Making predictions on the original test set...")
    # We use X_test_selected_scaled!! Test set should reflect real-world distribution.
    y_pred_smote = rf_model_smote.predict(X_test_selected_scaled)

    # 5. Evaluate the performance
    print("Evaluating model performance with SMOTE training data...")
    accuracy_smote = accuracy_score(y_test, y_pred_smote)
    print(f"\nAccuracy with SMOTE: {accuracy_smote:.4f}")

    # Detailed Classification Report
    print("\nClassification Report (with SMOTE):")
    # Get sorted unique labels from the original test set
    class_labels_smote = sorted(pd.Series(y_test).unique())
    try:
        report_smote = classification_report(y_test, y_pred_smote, zero_division=0, labels=class_labels_smote)
        print(report_smote)
    except Exception as e:
        print(f"Could not generate detailed report: {e}")
        report_smote = classification_report(y_test, y_pred_smote, zero_division=0)
        print(report_smote)


    # Confusion Matrix
    print("\nConfusion Matrix (with SMOTE):")
    cm_smote = confusion_matrix(y_test, y_pred_smote, labels=class_labels_smote)
    print(cm_smote) 

    print(f"Generating Confusion Matrix plot (saving to {image_dir}/confusion_matrix_rf_smote.png)...")
    plt.figure(figsize=(10, 8)) 
    sns.heatmap(cm_smote, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels_smote, yticklabels=class_labels_smote)
    plt.xlabel('Predicted Label') 
    plt.ylabel('True Label') 
    plt.title('Confusion Matrix - RandomForest (SMOTE)') 
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout() 
    plt.savefig(os.path.join(image_dir, 'confusion_matrix_rf_smote.png'), bbox_inches='tight')
    plt.close() 
    print("Confusion Matrix plot saved.")

except ValueError as e:
    print(f"\nSMOTE could not be applied. Error: {e}")
    print("This might happen if a class has fewer samples than k_neighbors (default 5).")
    print("Skipping SMOTE re-training and evaluation.")


Applying SMOTE to the training set...
Class distribution BEFORE SMOTE:
CANCER_TYPE_DETAILED
Breast Invasive Ductal Carcinoma     546
Breast Invasive Lobular Carcinoma    141
Breast Invasive Carcinoma (NOS)       52
Other_Subtype                         17
Name: count, dtype: int64

SMOTE applied successfully.
X_train shape AFTER SMOTE: (2184, 1016)
Class distribution AFTER SMOTE:
CANCER_TYPE_DETAILED
Breast Invasive Ductal Carcinoma     546
Breast Invasive Lobular Carcinoma    546
Breast Invasive Carcinoma (NOS)      546
Other_Subtype                        546
Name: count, dtype: int64

Training RandomForest model with SMOTE data...
Training completed.
Making predictions on the original test set...
Evaluating model performance with SMOTE training data...

Accuracy with SMOTE: 0.7908

Classification Report (with SMOTE):
                                   precision    recall  f1-score   support

  Breast Invasive Carcinoma (NOS)       0.25      0.04      0.07        23
 Breast Invasive